In [1]:
import os 
os.chdir("..")

In [69]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
from scipy import stats
import json

from config import DEVICE_SKIP_STATE, NUMERIC_CTX_SUFFIX
root_dir = "data/Thermostat Dataset"
processed_dir = "processed"
datetime_temp = "%Y-%m-%d %H:%M"
all_csv = [f for f in os.listdir(root_dir) if f.endswith(".csv") \
            and ("weather" not in f) and ("RB" not in f)]
weather_file = "weather.csv"
weathers = pd.read_csv(os.path.join(root_dir, weather_file))
# indoor_temp_matching = lambda x : any(["RMT_TL", ""])
col_idx = {
    "Time": 0,
    "Window": 1,
    "Occupancy": 2,
    "IndoorTemp": 3,
    "IndoorHumidity": 4,
    "IndoorCo2": 5,
    "Setpoint": 6,
    "OutTemp": 7,
}
ctx_to_include = [
    "OutTemp", "IndoorTemp", "IndoorHumidity",
]
outtemp_data = weathers.iloc[:,1]
print(min(outtemp_data), max(outtemp_data))
for filename in all_csv:
    file_path = os.path.join(root_dir, filename)
    data = pd.read_csv(file_path)
    data["OutTemp"] = outtemp_data
    occupied_idx = data.iloc[:, col_idx["Occupancy"]] >= 1
    print(filename, len(data))
    # setpoint_data = data.iloc[:, col_idx["Setpoint"]][occupied_idx]
    # humid_data = data.iloc[:, col_idx["IndoorHumidity"]][occupied_idx]
    # indoorTemp_data = data.iloc[:, col_idx["IndoorTemp"]][occupied_idx]

    device_evt = {"thermostat":[]}
    ctx_evt = {
        x:[]
        for x in ctx_to_include
    }
    prev_setpoint = None
    prev_state = DEVICE_SKIP_STATE
    for idx, row in data.iterrows():
        setpoint = row[col_idx["Setpoint"]]
        time = row[col_idx["Time"]]
        cur_time = datetime.strptime(time, datetime_temp)
        occupancy = row[col_idx["Occupancy"]]
        if occupancy == 0:
            # No one in the room
            if prev_state != DEVICE_SKIP_STATE:
                device_evt["thermostat"].append((DEVICE_SKIP_STATE, cur_time))
                prev_state = DEVICE_SKIP_STATE
                prev_setpoint = None
        else:
            if prev_setpoint != setpoint:
                prev_state = "auto"
                prev_setpoint = setpoint
                device_evt["thermostat"].append((prev_state, cur_time, setpoint))
    time_data = list(data.iloc[:, col_idx["Time"]])
    for ctx in ctx_to_include:
        data_col = list(data.iloc[:, col_idx[ctx]])
        for i, value in enumerate(data_col):
            if len(ctx_evt[ctx]) == 0 or \
                value != ctx_evt[ctx][-1][0]:
                cur_time = datetime.strptime(time_data[i], datetime_temp)
                ctx_evt[ctx].append((value, cur_time))
    

    ctx_evt = { k + NUMERIC_CTX_SUFFIX : v for k,v in ctx_evt.items()}
    with open(os.path.join(root_dir, processed_dir, filename), 'w') as f:
        f.write(json.dumps((ctx_evt, device_evt), default=str))
    # print(device_evt)
    # if len(setpoint_data) > 10000:
    #     print(len(setpoint_data))
    #     print(setpoint_data.corr(b_data))
    # print(indoorTemp_data.head())

-23.84985161 33.6
HS3301.csv 35041
